In [1]:
#Load some packages to be used in this program
import matplotlib.pyplot as plt
import scipy  as sp
import numpy  as np
import xarray as xr
import pyart  as pyart
import os
import glob
import time
import cv2
import imageio
from imgpy import Img
import pickle
import PIL
from PIL import Image
import sys
import moviepy.editor as mp
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/home/sransom/pyart/pyart/graph/cm.py:104: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:
/home/sransom/pyart/pyart/graph/cm_colorblind.py:32: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:


In [2]:
# Make function to crop image
def crop(img,box):
    with Img(fp=img) as im:
        im.crop(box=box)
        im.save(fp=img[0:-4] + '_cropped.gif')

#make function to convert to avi:
def gif2avi(gifpath):
    clip = mp.VideoFileClip(gifpath)
    clip.write_videofile(gifpath[0:-4] + ".avi", codec = 'rawvideo')
    avipath = gifpath[0:-4] + ".avi"
    return avipath

#make function to turn resize gifs such that they are the same dimensions as the masks
def resize(imagepath,dim1,dim2):
    image = Image.open(imagepath)
    image = image.resize((dim1, dim2), PIL.Image.ANTIALIAS)
    image.save(imagepath)
    resized_image = imageio.read(imagepath)
    return resized_image

In [4]:
#Import gifs
reflectivity_gif =   '/home/sransom/pyth/EG_Utqiagvik_gridded/sans_ice/reflectivity.gif'
#total_power_gif  =   '/home/sransom/pyth/EG_Utqiagvik_gridded/total_power.gif'
reflectivity_gifx2 = '/home/sransom/pyth/EG_Utqiagvik_gridded/sans_ice/reflectivity x2.gif'

#Crop gifs
box_top_left= (109, 87)
box_btm_rght = (643,486)
box = box_top_left + box_btm_rght

#Crop the images
crop(reflectivity_gif,box)
#crop(total_power_gif,box)
crop(reflectivity_gifx2,box)


#convert to avis:
reflectivity_cropped   = gif2avi("/home/sransom/pyth/EG_Utqiagvik_gridded/sans_ice/reflectivity_cropped.gif"   )
#total_power_cropped    = gif2avi("/home/sransom/pyth/EG_Utqiagvik_gridded/sans_ice/reflectivity_cropped.gif"   )
reflectivity_croppedx2 = gif2avi("/home/sransom/pyth/EG_Utqiagvik_gridded/sans_ice/reflectivity x2_cropped.gif")

t:  35%|███▌      | 84/240 [00:00<00:00, 836.87it/s, now=None]

Moviepy - Building video /home/sransom/pyth/EG_Utqiagvik_gridded/sans_ice/reflectivity_cropped.avi.
Moviepy - Writing video /home/sransom/pyth/EG_Utqiagvik_gridded/sans_ice/reflectivity_cropped.avi



Moviepy - Done !
Moviepy - video ready /home/sransom/pyth/EG_Utqiagvik_gridded/sans_ice/reflectivity_cropped.avi


Moviepy - Building video /home/sransom/pyth/EG_Utqiagvik_gridded/sans_ice/reflectivity x2_cropped.avi.
Moviepy - Writing video /home/sransom/pyth/EG_Utqiagvik_gridded/sans_ice/reflectivity x2_cropped.avi



Moviepy - Done !
Moviepy - video ready /home/sransom/pyth/EG_Utqiagvik_gridded/sans_ice/reflectivity x2_cropped.avi


In [ ]:
#Every time a new point starts being tracked, run this def to add it to a directory of its history
def add_point_directory(point_directory,new_points,tiempo):
    PDlen = len(point_directory)
    for pnum in range(PDlen, PDlen + len(new_points)-1):
        point_directory['point' + str(pnum)] = {
            'coords'  : [],
            'last_loc': [],
            'time'    : [],
            'u'       : [],
            'v'       : [],
            '__doc__' : \
                "'coords' is a 2-element list ('[xnew, ynew]') giving the current coordinates of the optical " \
                "flow program's best guess of the new location of the feature being tracked." \
                "'last_loc' is a 2-element list ([xold, yold]) giving the the coordinates of the feature from " \
                "the last frame which the optical flow algorithm has tracked to the location specified by 'coords'." \
                "\n 'time' is the time, in minutes, since the first frame." \
                "\n 'u' and 'v' are the mean zonal and meridional velocities (km/hr), respectively, of the feature " \
                "\n between the last point ([xold, yold]) and the latest point ([xnew, ynew])." 
        }

        (x, y) = new_points[pnum-PDlen][0]
        point_directory['point' + str(pnum)]['coords'].append([x, y])
        point_directory['point' + str(pnum)]['time'].append(tiempo)
    return point_directory

#Make def to calculate distance between two points
def dist(p1,p2):
    distance = np.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)
    return distance


In [ ]:
point_directory.keys()

In [ ]:
file = reflectivity_cropped
location = '/home/sransom/pyth/EG_Utqiagvik_gridded/sans_ice/OF_frames/'
borderimage = imageio.imread('/home/sransom/pyth/stationary mask boundary 399x534.png')
maskborder = np.asarray(borderimage)[:,:,0:3]
landshape = imageio.imread('/home/sransom/pyth/utqiagvik mask 399x534.png')[:,:,0:3]

In [ ]:
#calculate which pixel the boundary starts at 
#define a function to create a mask based on this png
def find_boundary(im):
    width =  np.shape(im)[1]        # get borderimageage size
    height =  np.shape(im)[0]
    notborder = im[0,0]   # find color that corresponds with not the border
    border = []
    for y in range(height):
        for x in range(width):
            if im[y,x] == notborder:
                continue  # mask on the point if it is ocean
            else:
                border.append(width)
    return border
coast = find_boundary(borderimage[:,:,0])


In [ ]:
#def optical_flow_Lucas_Kanade(file,location):
cv2.destroyAllWindows()

cap = cv2.VideoCapture(file)

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 100 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors to demarkate specific points
color = np.random.randint(0,255,(100,3))

#create empty point directory to add to later
point_directory = {}
#create dictionary storing individual frames:
animated_frames = {
    'with_ice'    : {},
    'just_tracks' : {}
}

# Take first frame and find corners in it
print('Finding features to track.')
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# Define first set of tracking features' coords:
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Add these initial coords to the point_directory:
point_directory = add_point_directory(point_directory,p0,0)
print(str(len(p0)) + ' starting features found.')

points_used_this_timestep = [len(p0)]

# Create an ice tracks image for drawing purposes
tracks = np.zeros_like(old_frame)

#flag that the point_adding algorithm has not been applied yet:
its_new_points = False

feature_lines = []
#while(1):
for count in range(1,119):
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    try:
        good_old = p0[st==1]
        good_new = p1[st==1]
        
    except TypeError:
        raise Warning("If you're seeing this error, your (all points moved off-screen) catchsafe probably isn't working")
        if count != 118:
            print('All features moved off-screen, adding ' + str(len(p0)) + ' new features.')
            p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
            point_directory = add_point_directory(point_directory,p0,count*15)
        continue
    points_used_in_this_loop = []
    # draw the tracks
    my_frame = frame.copy()
    for (new,old) in zip(good_new,good_old):
        try:
            xold, yold = old.ravel()
            xnew, ynew = new.ravel()
            
            #find the number this point is indexed as 
            already_IDed = False #reset ambiguity check
            if its_new_points:
                contenders = range(old_dic_length)
            else:
                contenders = range(len(point_directory))
            for contendernum in contenders:
                try:
                    contender_coords = point_directory['point' + str(contendernum)]['coords'][-1]
                except IndexError:
                    pointID = contendernum
                if contender_coords == [xold, yold]:
           #         if already_IDed:
           #             raise Warning('point ' + str(contendernum) + ' is ambiguous with a previous point: ' + str(pointID))
           #             del point_ID #ensure that if no point is IDed that it doesn't just register as the last point
                    pointID = contendernum
                    already_IDed = True #Mark this count's point as already ID'ed
            its_new_points = False #indicate that these aren't new points this round
            
            point_directory['point' + str(pointID)]['coords'].append([xnew, ynew])
            point_directory['point' + str(pointID)]['last_loc'].append([xold, yold])
            point_directory['point' + str(pointID)]['time'].append(count*15)
            points_used_in_this_loop.append(pointID)
            tracks = cv2.line(tracks, (xold, yold), (xnew, ynew), tuple(color[pointID].tolist()), 2)
        except AttributeError:
            continue
        #tracks = cv2.circle(tracks,(a,b),5,color[i].tolist(),-1)
    
    frame[frame==255.] = 0
    img = cv2.add(frame,tracks)
    img = frame + tracks + maskborder + landshape
    just_tracks = tracks + maskborder + landshape
    animated_frames['with_ice']['frame' + str(count)]:img
    animated_frames['just_tracks']['frame' + str(count)]:just_tracks

    cv2.imshow('frame',just_tracks)
    cv2.imwrite(location + 'reflectivity' + str(count) + '.png',just_tracks)
    k = cv2.waitKey(30) & 0xff
    #print(k)
    if k == 27:
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
    
    print(len(p0))

    #add points if the program is running low on points
    p0len = len(p0)
    if p0len < 6: 
        old_dic_length = len(point_directory)
        pnew = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
        #check if new point is too near an already discovered point (i.e. not a new point); make index of valid new points
        valid_pnews = np.ones(len(pnew),dtype = bool)
        for remaining_point in range(p0len):
            for new_point in range(len(pnew)):
                if dist(pnew[new_point,:][0],p0[remaining_point,:][0]) < 10: #check if dist b/w points is < 10px
                    valid_pnews[new_point] = False
        #select just valid pnews:
        pnew = pnew[valid_pnews]
        p0 = np.asarray(tuple(p0) + tuple(pnew))
        
        #Add new points' data to the point directory
        point_directory = add_point_directory(point_directory,pnew,count*15)
        
        
        #flag that these are new points
        its_new_points = True
        print('Fewer than 5 points remaining after ' + str(count) + ' iterations; added ' + str(len(pnew)) + ' new features.')
    print('p0 length = ' + str(len(p0)) + ' after ' + str(count) + ' iterations; points used this loop: ' + str(points_used_in_this_loop))
    points_used_this_timestep.append(len(points_used_in_this_loop))

    #cv2.destroyAllWindows()
cap.release()
print('yeet')

###### ffmpeg -f image2 -r 1/5 -i image%05d.png -vcodec mpeg4 -y movie.mp4

In [ ]:
point_directory['point' + str(contendernum)]['coords']

In [ ]:
point_directory_save_location = open("/home/sransom/pyth/point_directory.pkl","wb")
pickle.dump(point_directory,point_directory_save_location)
point_directory_save_location.close()

In [ ]:
print(point_directory['point0']['__doc__'])

In [ ]:
#calculate distances between latitudes
latrange = [-157.0, -156.2]
lonrange = [71.2, 71.5]
latrangemeters = np.sin(np.diff(latrange) * np.pi/180)*6.371e6
lonrangemeters = np.sin(np.diff(lonrange) * np.pi/180)*6.371e6
resolution = 401
#define conversion factor from px to meters
px_per_m_lat = resolution/latrangemeters
px_per_m_lon = resolution/lonrangemeters
#define conversion factor from pixels per second to km perhour
pxps2metersph_lat = 3600/px_per_m_lat
pxps2metersph_lon = 3600/px_per_m_lon

In [ ]:
#make motion vectors from coords listed in point_directory
for pnum in range(len(point_directory.keys())):
    #extract this point's data:
    point = point_directory['point' + str(pnum)]
    #find initial position and time of this point:
    x0, y0 = point['coords'][0]
    time0  = point['time'][0]*3600
    #reset feature position index
    coordnum = 0
    #reset velocities for this point so that we aren't just adding new extraneous data:
    point_directory['point' + str(pnum)]['u'] = []
    point_directory['point' + str(pnum)]['v'] = []
    #begin looping through this point's coordinates:
    while(1):
        try:
            x1, y1 = point['coords'][coordnum]
            time1  = point['time'][coordnum]*3600
            
            #calculate u and v vectors and register them in the point directory:
            point_directory['point' + str(pnum)]['u'].append((x1-x0)/(time1-time0)*pxps2metersph_lat)
            point_directory['point' + str(pnum)]['v'].append((y1-y0)/(time1-time0)*pxps2metersph_lon)
            
            #reset indices for next loop iteration
            x0, y0, time0 = (x1, y1, time1)
            coordnum += 1
            
            
        #once last tracked position is reached, break to next point:
        except IndexError: 
            break



In [ ]:
def lobf(this_axis):
    velnum = np.zeros(120)
    velsum = np.zeros(120)
    for lineas in this_axis.get_lines():
        xdata = lineas.get_xdata()
        ydata = lineas.get_ydata()
        #timeindex = np.array([int(x/15) for x in xdata])
        for i, a in enumerate(xdata):
            velnum[int(a/15)] += float(1)
            velsum[int(a/15)] += float(ydata[i])
    lobf = velsum/velnum
    return lobf
        #velnum[timeindex] += 1
        #velsum[timeindex] += ydata

In [ ]:
len(lobf(axes[0]))

In [ ]:
120*15

In [ ]:
np.arange(0,60,15)

In [ ]:
#plot velocities
higo = plt.figure()
higo, axes = plt.subplots(2,1,figsize = (20,20))
plt.subplots_adjust(hspace = .2)
for key in point_directory.keys():
    point = point_directory[key]
    axes[0].plot(point['time'],point['u'], linewidth = 2)
    axes[0].set_title('Zonal velocities')
    axes[0].set_ylabel('Velocity (m/hr)')
    axes[0].set_ylim(-200,200)
    axes[0].set_xlim(0,1750)
    axes[0].set_xlabel('time (minutes)')
    axes[1].plot(point['time'],point['v'], linewidth = 2)
    axes[1].set_title('Meridional velocities')
    axes[1].set_ylabel('Velocity (m/hr)')
    axes[1].set_ylim(-50,50)
    axes[1].set_xlim(0,1750)
    axes[1].set_xlabel('time (minutes)')
axes[0].plot(np.arange(0,1800,15),lobf(axes[0]),linewidth = 4, color = 'k')
axes[1].plot(np.arange(0,1800,15),lobf(axes[1]),linewidth = 4, color = 'k')
higo.savefig('/home/sransom/pyth/papergraphics/velocities.png', bbox_inches = 'tight')
    #find mean velocities in each array:

In [ ]:
#plot number of points plotted by timestep
higo = plt.figure()
plt.plot(points_used_this_timestep)
plt.xlabel('Timestep')
plt.ylabel('Number of features being tracked')
plt.suptitle('Number of points being tracked \n by optical flow per timestep: \n')
higo.savefig('/home/sransom/pyth/papergraphics/points_tracked.png', bbox_inches = 'tight')

In [ ]:
mygrid = pyart.io.read_grid('/home/sransom/pyth/EG_Utqiagvik_gridded/grids/BRW190120001204.nc')

In [ ]:
egdata = np.ma.getdata(mygrid.fields['reflectivity']['data'])
egdata = egdata.ravel()
egdata = egdata[~np.isnan(egdata)]

plt.hist(egdata, bins= np.linspace(10,40,30), rwidth = .9, density = True)
plt.xlabel('Equivalent reflectivity factor (dBZ)')
plt.ylabel('Percentage of data in range')
plt.suptitle('PDF of reflectivity values excluding land values')
plt.savefig('/home/sransom/pyth/papergraphics/power_spectrum_sans_land.png', bbox_inches = 'tight')

In [ ]:
raise ValueError('no need to run past here if running whole notebook. type Ctrl+f+yeeet to find this point')

#def optical_flow_Lucas_Kanade(file,location):
cv2.destroyAllWindows()

cap = cv2.VideoCapture(file)

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 100 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

#create empty point directory to add to later
point_directory = {}

# Take first frame and find corners in it
print('Finding features to track.')
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
point_directory = add_point_directory(point_directory,p0)
print(str(len(p0)) + ' starting features found.')

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

feature_lines = []
#while(1):
for count in range(119):
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

   # print(count)
   # print(p1)
   # print(st)
    #try:
    # Select good points
    good_new = good_old = np.zeros((len(st),2),dtype = np.float32)
    for yeet in range(len(st)):
        if st[yeet] == 1:
            good_new[yeet] = p1[yeet]
            good_old[yeet] = p0[yeet]
        else:
            good_old[yeet] = 0
            good_new[yeet] = 0
        ##If the results of attempting the above is just too abominable, just replace it with the following:
        #good_new = p1[st==1]
        #good_old = p0[st==1]
   # except TypeError:
   #     print('All features moved off-screen, adding ' + str(len(p0)) + ' new features.')
   #     p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
   #     point_directory = add_point_directory(point_directory,p0)
   #     continue
    p1len = len(p1[p1!=0])
    if p1len < 3: #add points if the program is running low on points
        pnew = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
        #check if new point is too near an already discovered point (i.e. not a new point); make index of valid new points
        valid_pnews = np.ones(len(pnew),dtype = bool)
        for remaining_point in range(p1len-1):
            for new_point in range(len(pnew)-1):
                try:
                    if dist(pnew[new_point,:],p1[p1!=0][remaining_point,:]) < 10: #check if dist b/w points is < 10px
                        valid_pnews[new_point] = False
                except IndexError:
                    if dist(pnew[new_point,:][0],p1[p1!=0]) < 10: #check if dist b/w points is < 10px
                        valid_pnews[new_point] = False        #select just valid pnews:
        pnew = pnew[valid_pnews]
        p1 = np.asarray(tuple(p1.ravel()) + tuple(pnew.ravel()))
        point_directory = add_point_directory(point_directory,p1)
        print('Fewer than 3 points remaining after ' + str(count) + ' iterations; added ' + str(len(pnew)) + ' new features.')

    # draw the tracks
    my_frame = frame.copy()
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        try:
            a,b = new.ravel()
            point_directory['point' + str(i)].append([a,b])
        except AttributeError:
            continue
        try:
            c,d = old.ravel()
            point_directory['point' + str(i)].append([c,d])
        except AttributeError:
            continue
        mask = cv2.line(mask, (a,b),(c,d), tuple(color[i].tolist()), 2)
        mask = cv2.circle(mask,(a,b),5,color[i].tolist(),-1)
    frame[frame==255.] = 0
    img = cv2.add(frame,mask)

    cv2.imshow('frame',img)
    cv2.imwrite(location,img)
    k = cv2.waitKey(30) & 0xff
    #print(k)
    if k == 27:
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

#cv2.destroyAllWindows()
cap.release()
print('yeet')

In [ ]:
cv2.destroyAllWindows()

#def optical_flow_Lucas_Kanade(file,location):
cap = cv2.VideoCapture(file)

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 100 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

#create empty point directory to add to later
point_directory = {}

# Take first frame and find corners in it
print('Finding features to track.')
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
add_point_directory(point_directory,p0)
print(str(len(p0)) + ' starting features found.')

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

feature_lines = []
#while(1):
for count in range(119):
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

   # print(count)
   # print(p1)
   # print(st)
    try:
        # Select good points
        good_new = p1[st==1]
        good_old = p0[st==1]
    except TypeError:
        print('All features moved off-screen, adding ' + str(len(p0)) + ' new features.')
        p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
        add_point_directory(point_directory,p0)
        continue

    if len(p1) < 3: #add points if the program is running low
        pnew = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
        p0 = np.asarray(tuple(p0.ravel()) + tuple(pnew.ravel()))
        print('Fewer than 3 points remaining after ' + str(count) + ' iterations...added ' + str(len(pnew)) + ' new features.')

    show_ice = True
        
    # draw the tracks
    if show_ice:
        for i,(new,old) in enumerate(zip(good_new,good_old)):
            a,b = new.ravel()
            c,d = old.ravel()
            mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
            frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
        frame[frame==255.] = 0
        img = cv2.add(frame,mask)
    else:
        my_frame = frame.copy()
        for i,(new,old) in enumerate(zip(good_new,good_old)):
            a,b = new.ravel()
            c,d = old.ravel()
            mask = cv2.line(mask, (a,b),(c,d), tuple(color[i].tolist()), 2)
            #mask = cv2.circle(mask,(a,b),5,color[i].tolist(),-1)
        img = mask

    cv2.imshow('frame',img)
    #cv2.imwrite(location,img)
    k = cv2.waitKey(30) & 0xff
    #print(k)
    if k == 27:
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

#cv2.destroyAllWindows()
cap.release()
print('yeet')

In [ ]:
point_directory.keys()

In [ ]:
location = '/home/sransom/pyth/EG_Utqiagvik_gridded/sans_ice/optical_flow_reflectivity x2.png'
optical_flow_Lucas_Kanade(reflectivity_cropped,location)

In [ ]:
raise ValueError('Terminate')

In [ ]:
def optical_flow_dense_method(file):
    cap = cv2.VideoCapture(file)

    ret, frame1 = cap.read()
    prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    hsv = np.zeros_like(frame1)
    hsv[...,1] = 255

    while(1):
        ret, frame2 = cap.read()
        next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

        flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
        hsv[...,0] = ang*180/np.pi/2
        hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
        rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)

        cv2.imshow('frame2',rgb)
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
        elif k == ord('s'):
            cv2.imwrite('/home/sransom/opticalfb.png',frame2)
            cv2.imwrite('/home/sransom/opticalhsv.png',rgb)
        prvs = next

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
optical_flow_dense_method(total_power_cropped)

In [ ]:
radar = pyart.io.read('/nfs/gce/projects/digr/xsapr_nsa/BRW190120002403.RAWTA32.maint')

In [ ]:
radar.range